In [2]:
#Libraries for the Project

import pandas as pd
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split, KFold, cross_val_score, cross_validate, GridSearchCV,RandomizedSearchCV
import numpy as np
from matplotlib import pyplot as plt
from scipy.stats import zscore
from sklearn.preprocessing import StandardScaler

INFO 698 Notebook
Created 10/7/2025

Academic Data Analysis Capstone Project

In [ ]:
import pandas as pd
raw_data = pd.read_csv('DataSets/RawData.csv')

#print(raw_data.head())

Data Preparation:
We wil clean and modify the data set in preparation for bulding our models

In [ ]:
#load dataset with null Term GPA excluded
data_set = pd.read_csv('DataSets/RawDataWithStudentsRemoved.csv')

print(data_set.head())

#data_set.dtypes
#data_set.shape


   FakeIdentifier  Age  Gender PrimaryMilitaryAffiliation AcademicCareer  \
0               1   18  Female    No Military Affiliation  Undergraduate   
1               2   72    Male    No Military Affiliation            Law   
2               3   26    Male    No Military Affiliation       Graduate   
3               4   47  Female                    Veteran       Graduate   
4               5   45    Male    No Military Affiliation  Undergraduate   

                          College  NumberofClassesEnrolled  TermGPA  \
0              College of Science                        4     3.75   
1   James E Rogers College of Law                        3     4.00   
2          College of Engineering                        2      NaN   
3       Coll of Ag Life & Env Sci                        2     4.00   
4  College of Information Science                        2     2.00   

   CumulativeGPA  UnitsPassedincludedinGPA  UnitsPassednotincludedinGPA  \
0          3.636                         

(63398, 15)

In [ ]:

data_set_nulls = data_set.isnull().sum()
print(data_set_nulls)

In [15]:
data_set = data_set.dropna(subset = ['TermGPA'])



In [ ]:
#no nulls
data_set_nulls = data_set.isnull().sum()
print(data_set_nulls)

In [ ]:
#mean and standard dev by College

#mean_by_college = data_set.groupby('College')['TermGPA'].mean()
#print(mean_by_college)

stdev_by_college = data_set.groupby('College')['TermGPA'].std()
print(stdev_by_college)

In [ ]:
#Split the Data Set

# Y target column
target_column = 'TermGPA'
test_percent = 0.20

#x_columns_to_drop = ['FakeIdentifier' , 'TermGPA']

#data set dropping the target feature
x_features = data_set.drop(['FakeIdentifier' ,
                             #'College',
                             'CumulativeGPA',
                             #'UnitsPassedincludedinGPA',
                             'TermGPA'],
                               axis = 1)


#encoding categorical features
columns_to_encode = ['Gender',
                      'PrimaryMilitaryAffiliation',
                        'AcademicCareer',
                          'College',
                      'UAFullTimePartTime',
                        'AcademicLevelEndofTerm',
                          'FirstGenerationFlag' 
                          ]


y_target = data_set[target_column]


#split the data into a 80/20 train test split using the same random seed for consistency
x_train, x_test, y_train, y_test = train_test_split(x_features, y_target, test_size = test_percent, random_state = 698) 

#encode categorical features, and drop one level to avoid multicolinearity
x_train_encoded = pd.get_dummies(x_train,
                                  columns=columns_to_encode,
                                    drop_first=True,
                                      dtype = int)
x_test_encoded = pd.get_dummies(x_test,
                                 columns=columns_to_encode,
                                 drop_first=True,
                                   dtype = int)
# restore missing features after encoding
x_train_features_encoded_cols = x_train_encoded.columns
x_test_encoded = x_test_encoded.reindex(columns=x_train_features_encoded_cols, fill_value=0)

In [ ]:
#binning the target into quartiles

quantile_labels = ['Q1','Q2','Q3','Q4']

bins = pd.qcut(y_train, q=4, labels=quantile_labels, retbins=True, duplicates='drop')

y_train_binned = pd.cut(y_train, bins=bins, include_lowest=True, labels=quantile_labels)

#bin y test wiht the same bins as y train
y_test_binned = pd.cut(y_test, bins=bins, include_lowest=True, labels=quantile_labels)

In [ ]:
#notnormally distributed

import matplotlib.pyplot as plt
import seaborn as sns

 
sns.histplot(data_set['CumulativeGPA'], kde=True)
plt.title('Histogram of TermGPA')
plt.show()

In [ ]:
# the RF Calssifier model

metric = roc_auc_score
metric_name = "ROC AUC"
rfc_depth = 15
rfc_max_features = None # None, sqrt or log2
rfc_min_sample_split = 4
#rfc_criterion = 'squared_error' #criterion{“squared_error”, “absolute_error”, “friedman_mse”, “poisson”}, default=”squared_error”
rfc_estimators = 100

model = RandomForestClassifier(n_estimators = rfc_estimators,
                                 max_features = rfc_max_features,
                                    min_samples_split=rfc_min_sample_split,
                                      max_depth = rfc_depth,
                                          random_state=698)


In [ ]:
#fit the model
model.fit(x_train_encoded, y_train_binned)

# Predict using test data
y_pred = model.predict(x_test_encoded) 

    # compare prediction to actual dev/test data
score = metric(y_test_binned, y_pred)
print(f"{metric_name} on test set: {score:.4f}") 

#r_squared = r2_score(y_test, y_pred)
#print(f"r_squared on test set: {r_squared:.4f}") 

In [ ]:
#add mean and standard dev by College to dataframe

#data_set['meanTermGPACol'] = data_set.groupby('College')['TermGPA'].transform('mean')
#data_set['stdevTermGPACol'] = data_set.groupby('College')['TermGPA'].transform('std')
#data_set['ZTermGPA'] = data_set.groupby('College')['TermGPA'].transform(zscore)

#print(data_set[['FakeIdentifier', 'TermGPA','College' ,'meanTermGPACol','stdevTermGPACol', 'ZTermGPA' ]].head())

In [ ]:
#create classes from normalized Term GPA
# very low z < -1.5
#low -1.5 to -0.5
#average -0.5 to 0.5
#above average 0.5 to 1.5
#very above avg > 1.5

#data_set['ClassTermGPA'] = ['WellBelowAverage' if 'ZTermGPA' < -1.5
                               #else 'BelowAverage' if ('ZTermGPA' < -0.5 & 'ZTermGPA' >= -1.5)
                              # else 'Average' if ('ZTermGPA' >=-0.5 & 'ZTermGPA' < 0.5)
                              # else 'AboveAverage' if 'ZTermGPA' >= 0.5 & 'ZTermGPA' < 1.5
                               #else 'WellAboveAverage' if 'ZtermGPA' >=1.5
                               #else 'NA']



#data_set['ClassTermGPA'] = ['WellBelowAverage' if ztermgpa < -1.5
                               #else 'BelowAverage' if (ztermgpa < -0.5 and ztermgpa >= -1.5)
                               #else 'Average' if (ztermgpa >=-0.5 and ztermgpa < 0.5)
                               #else 'AboveAverage' if ztermgpa >= 0.5 and ztermgpa < 1.5
                               #else 'WellAboveAverage' if ztermgpa >=1.5
                               #else 'NA' for ztermgpa in data_set['ZTermGPA']]

#print(data_set[['FakeIdentifier','ZTermGPA','ClassTermGPA']])

In [35]:
print(len(data_set))

55569


Setting Up the Random Forest Model
Test Train Split
Need to encode categorical features
